In [ ]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam

from zoo.pipeline.api.keras.layers import *
from zoo.pipeline.api.keras.models import *
from bigdl.optim.optimizer import *

EPISODES = 1000


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(learningrate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict_local(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict_local(next_state)[0]))
            target_f = self.model.predict_local(state)
            target_f[0][action] = target
#             print("State: " + str(state))
            self.model.fit(state, target_f, nb_epoch=10, batch_size=1, distributed=False)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
creating: createZooKerasSequential
creating: createZooKerasDense
creating: createZooKerasDense
creating: createZooKerasDense
creating: createAdam
creating: createMSECriterion
episode: 0/1000, score: 23, e: 1.0
episode: 1/1000, score: 19, e: 1.0
episode: 2/1000, score: 13, e: 0.99
episode: 3/1000, score: 16, e: 0.99
episode: 4/1000, score: 11, e: 0.99
episode: 5/1000, score: 29, e: 0.98
episode: 6/1000, score: 53, e: 0.98
episode: 7/1000, score: 20, e: 0.97
episode: 8/1000, score: 13, e: 0.97
episode: 9/1000, score: 13, e: 0.96
episode: 10/1000, score: 28, e: 0.96
episode: 11/1000, score: 14, e: 0.95
episode: 12/1000, score: 24, e: 0.95
episode: 13/1000, score: 27, e: 0.94
episode: 14/1000, score: 24, e: 0.94
episode: 15/1000, score: 17, e: 0.93
episode: 16/1000, score: 63, e: 0.93
episode: 17/1000, score: 10, e: 0.92
episode: 18/1000, score: 11, e: 0.92
episode: 19/1000, score: 11, e: 0.9

In [3]:

from zoo.pipeline.api.keras.layers import *
from zoo.pipeline.api.keras.models import *

model = Sequential()
model.add(Dense(4, activation="relu", input_shape=(10, )))
x = np.random.random([300, 10])
y = np.random.random([300, ])
model.compile(optimizer="sgd", loss="mae")
model.fit(x, y, batch_size=112, nb_epoch=2)
result = model.predict(x)
print(result.collect())

creating: createZooKerasSequential
creating: createZooKerasDense
creating: createDefault
creating: createSGD
creating: createAbsCriterion
[array([ 0.        ,  0.26820531,  0.19068582,  0.        ], dtype=float32), array([ 0.18965068,  0.14505336,  0.66127777,  0.        ], dtype=float32), array([ 0.        ,  0.3243821 ,  0.33596537,  0.        ], dtype=float32), array([ 0.50129622,  0.        ,  0.74354017,  0.        ], dtype=float32), array([ 0.03218384,  0.41524306,  0.        ,  0.        ], dtype=float32), array([ 0.        ,  0.20426211,  0.56106591,  0.        ], dtype=float32), array([ 0.        ,  0.        ,  0.25241271,  0.        ], dtype=float32), array([ 0.        ,  0.46623424,  0.85750449,  0.        ], dtype=float32), array([ 0.3538869 ,  0.12055537,  0.72213173,  0.        ], dtype=float32), array([ 0.12228503,  0.        ,  0.21839646,  0.        ], dtype=float32), array([ 0.,  0.,  0.,  0.], dtype=float32), array([ 0.        ,  0.        ,  0.98139375,  0.        